In [33]:
import requests
import bz2

In [34]:
# 1. Get data from url, decompress and save file
DATA_URL = "https://gist.github.com/juanriaza/9b6f7ab3fc1cbda81100fa5d32512fd8/raw/a61db2d8d81789d5c7040eacee7b225bc2b2aa7f/data.csv.bz2"
r = requests.get(DATA_URL)
with open('data.csv.bz2', 'wb') as f:
    f.write(r.content)

with open('data.csv', 'wb') as new_file, bz2.BZ2File('data.csv.bz2', 'rb') as file:
        for data in iter(lambda : file.read(100 * 1024), b''):
            new_file.write(data)

In [61]:
# Importing resulting data into sqlite DB using command line

# I've created a schema because the default schema imports it
# everything as a string and makes it difficult to handle 
# numeric values

# Also the csv has a | as a separator so we have to specify it

# And we have to skip the first row (header) because the table
# already has it. Without this step the table would have
# the headers repeated
!sqlite3 main.db 'CREATE TABLE IF NOT EXISTS "properties"( \
  "customer_id" TEXT, \
  "city" TEXT, \
  "typology" TEXT, \
  "subtypology" TEXT, \
  "transaction_type" TEXT, \
  "price" REAL, \
  "zipcode" INTEGER, \
  "bedrooms" INTEGER, \
  "bathrooms" INTEGER, \
  "area" INTEGER, \
  "area_land" INTEGER, \
  "is_professional" INTEGER, \
  "date_added" TEXT \
);' '.mode csv' '.separator "|"' '.import --skip 1 data.csv properties'

In [136]:
# executing queries sqlite3 library
import sqlite3
con = sqlite3.connect("main.db")
cur = con.cursor()

# 1. Number of properties each professional client has in rental per ZIP code
res = cur.execute('select customer_id, zipcode, count(*) as counts from properties where transaction_type="rent" and is_professional=1 group by customer_id, zipcode order by customer_id desc, zipcode desc;')
res.fetchall()

[('393433303033313033373236', 8394, 4),
 ('393433303033313033373236', 8360, 1),
 ('393433303033313033373236', 8358, 2),
 ('393433303033313033373236', 8349, 1),
 ('393433303033313033373236', 8339, 1),
 ('393433303033313033373236', 8320, 1),
 ('393433303033313033373236', 8310, 1),
 ('393433303033313033373236', 8304, 6),
 ('393433303033313033373236', 8303, 16),
 ('393433303033313033373236', 8302, 44),
 ('393433303033313033373236', 8301, 17),
 ('393433303033303731313536', 8184, 2),
 ('393433303033303731313536', 8140, 1),
 ('393433303033303730393032', 8859, 5),
 ('393433303033303639393339', 8917, 1),
 ('393433303033303639393339', 8912, 3),
 ('393433303033303639393339', 8911, 1),
 ('393433303033303639393339', 8213, 1),
 ('393433303033303639393339', 8027, 1),
 ('393433303033303639393339', 8022, 1),
 ('393433303033303639343934', 8940, 2),
 ('393433303033303639343934', 8840, 1),
 ('393433303033303639343934', 8041, 2),
 ('393433303033303639343934', 8036, 1),
 ('393433303033303639343934', 8028, 1

In [271]:
# 2. Cheapest squared meter's zipcode

# For this exercise and the followinig one (its almost the same), I've made the following
# decisions:
# 1. price < 99999998.0 and price > 120: Super high values and values under 120 do not make
#    sense. Sometimes people just put ads with strange values but they put the correct value
#    in the ad's description.
# 2. (zipcode >= 8001 and zipcode <= 8980): Looks strange but there are empty and invalid range
#    zipcodes so we have to filter them.
# 3. area > 7: all properties with area < 7 do not make sense and make the price/m2 incoherent,
#    so we have to filter it out. From > 7 all properties do have correct areas.


res = cur.execute('select zipcode, (price/area) as ppm2 from properties where transaction_type="sale" and price < 99999998.0 and price > 120 and (zipcode >= 8001 and zipcode <= 8980) and area > 7 order by ppm2 asc limit 1;')
zipcode, price = res.fetchall()[0]
print(f'Cheapest zipcode is {zipcode} with {price}€/m2')

Cheapest zipcode is 8739 with 0.10344827586206896€/m2


In [272]:
# 3. Most expensive squared meter's zipcode
res = cur.execute('select zipcode, (price/area) as ppm2 from properties where transaction_type="sale" and price < 99999998.0 and price > 120 and (zipcode >= 8001 and zipcode <= 8980) and area > 7 order by ppm2 desc limit 1;')
zipcode, price = res.fetchall()[0]
print(f'Most expensive zipcode is {zipcode} with {price}€/m2')

Most expensive zipcode is 8358 with 173000.0€/m2


In [276]:
# 4. Professional client with the oldest property portfolio

# We can calculate the oldest property by making the average of the
# dates of every client's properties, or by making the sum. 
# Depending on that, it changes the results.


operation = "avg"
query = f'''select customer_id, {operation}(CAST(strftime('%s', date_added) as integer)) as avg_epoch from properties where date_added != "" and is_professional = 1 group by customer_id order by avg_epoch asc limit 1'''
res_avg = cur.execute(query)
cid_avg, _ = res_avg.fetchall()[0]

operation = "sum"
query = f'''select customer_id, {operation}(CAST(strftime('%s', date_added) as integer)) as avg_epoch from properties where date_added != "" and is_professional = 1 group by customer_id order by avg_epoch asc limit 1'''
res_sum = cur.execute(query)
cid_sum, _ = res_sum.fetchall()[0]


print(f'[AVG] Professional client with the olders portfolio is {cid_avg}')
print(f'[SUM] Professional client with the olders portfolio is {cid_sum}')

[AVG] Professional client with the olders portfolio is 39323032373531313531353331
[SUM] Professional client with the olders portfolio is 39323032373532363535363035


In [277]:
# 5. Which city is the most expensive developable land?
res = cur.execute('select city, price from properties where subtypology = "Terreno Urbanizable" and price < 99999998.0 order by price desc limit 1;')
city = res.fetchall()[0][0]
print(f'Most expensive city is {city}')

Most expensive city is Barcelona Capital


In [188]:
# 6. Do the same exercises but with pandas
import pandas as pd

# load csv
dtypes = {
    'customer_id': str,
    'city': str,
    'typology': str,
    'subtypology': str,
    'transaction_type': str,
    'price': int,
    'zipcode': str,
    'bedrooms': int,
    'bathrooms': int,
    'area': int,
    'area_land': int,
    'is_professional': bool,
    'date_added': str,
}
df = pd.read_csv('data.csv', delimiter="|", dtype=dtypes)


# 6.1.  Number of properties each professional client has in rental per ZIP code
condition = (df['transaction_type'] == "rent") & (df['is_professional'] == 1)
ndf = df[condition]
ndf = ndf.groupby(['customer_id', 'zipcode']).size().reset_index(name='counts')
ndf = ndf.sort_values(by=['customer_id', 'zipcode'], ascending=False)
ndf


,customer_id,zipcode,counts
6779,393433303033313033373236,08394,4
6778,393433303033313033373236,08360,1
6777,393433303033313033373236,08358,2
6776,393433303033313033373236,08349,1
6775,393433303033313033373236,08339,1
...,...,...,...
4,393030313430303030323137,08003,1
3,393030313430303030323137,08001,1
2,393030303830303030333431,08970,1
1,393030303830303030333431,08960,8


In [189]:
# 6.2. Cheapest squared meter's zipcode

# where transaction_type="sale" and price < 99999998.0 and price > 120 and (zipcode >= 8001 and zipcode <= 8980) and area > 7
condition = (df['transaction_type'] == "sale") & \
    (df['price'] < 99999998.0) & (df['price'] > 120) & \
    (df['area'] > 7)
ndf = df[condition]

# to apply the zipcode condition i will apply a function because andorra has a non-numerical
# postalcode and does not let us convert the column to int
def filter_zipcode(row):
    try:
        return 1 if 8001 <= int(row['zipcode']) <= 8980 else 0 # correct condition
    except:
        return 0
ndf = ndf[ ndf.apply(filter_zipcode, axis=1) == 1]

import numpy as np

# create new column for (price/area) as ppm2 (price per square metres)
ndf['ppm2'] = ndf['price']/ndf['area']
ndf = ndf.sort_values(by=['ppm2'])
print(ndf.iloc[0]['zipcode'])

08739


In [190]:
# 6.3 Most expensive squared meter's zipcode
print(ndf.iloc[ndf.shape[0]-1]['zipcode'])


08358


In [206]:
# 6.4 Professional client with the oldest property portfolio

# where date_added != "" and is_professional = 1
condition = (df['date_added'] != "") & \
    (df['is_professional'] == True) 
ndf = df[condition]

# group by customer_id order by avg_epoch asc 
#ndf = ndf.groupby(['customer_id'])

In [212]:
# 6.5. Which city is the most expensive developable land?

# where subtypology = "Terreno Urbanizable" and price < 99999998.0
condition = (df['subtypology'] == "Terreno Urbanizable") & \
    (df['price'] < 99999998.0) & (df['price'] > 0) 
ndf = df[condition].sort_values(by=['price'], ascending=False)
print(ndf.iloc[0]['city'])

Barcelona Capital


In [251]:
# 7. An investor wants to know which properties are interesting by postal code.
# Add to the pandas dataframe the following columns:
# over_valued (0/1)
# normal (0/1)
# under_valued (0/1)
#
# Which postal code has the most amound of bargains?

# To do this exercise, we will compute the average of the price/m2 per zipcode
# and compare the price of the property with the average per zipcode (with 
# a certain margin) to see if its over_valued, under valued or normal price

# using same filtering conditions as in 6.2
condition = (df['transaction_type'] == "sale") & \
    (df['price'] < 99999998.0) & (df['price'] > 120) & \
    (df['area'] > 7)
ndf = df[condition]
ndf = ndf[ ndf.apply(filter_zipcode, axis=1) == 1]
ndf

# add ppm2 column
ndf['ppm2'] = ndf['price']/ndf['area']

# calculate the mean of the ppm2 for each zipcode
ppm2_df = ndf.groupby(['zipcode'])['ppm2'].mean().reset_index(name="avg_ppm2")

# add extra columns
def normal(value: int, mean: float) -> int:

    margin = 0.07   # 7% error margin
    diff = value-mean
    if 0 <= diff <= margin*mean:
        return 1 # normal
    elif -margin*mean <= diff <= 0:
        return 1 # normal
    else:
        return 0
    
    
def over_valued(value: int, mean: float) -> int:
    margin = 0.07   # 7% error margin
    diff = value-mean
    
    if diff > margin*mean:
        return 1 # over_valued
    else:
        return 0
    
def under_valued(value: int, mean: float) -> int:
    margin = 0.07   # 7% error margin
    diff = value-mean
    
    if diff < -margin*mean:
        return 1 # under_valued
    else:
        return 0
    
    
ndf['over_valued'] = ndf.apply(
    lambda row: over_valued(
        row['price']/row['area'],
        ppm2_df[ ppm2_df['zipcode'] == row['zipcode'] ].iloc[0]['avg_ppm2']),
    axis=1) 

ndf['under_valued'] = ndf.apply(
    lambda row: under_valued(
        row['price']/row['area'],
        ppm2_df[ ppm2_df['zipcode'] == row['zipcode'] ].iloc[0]['avg_ppm2']),
    axis=1) 

ndf['normal'] = ndf.apply(
    lambda row: normal(
        row['price']/row['area'],
        ppm2_df[ ppm2_df['zipcode'] == row['zipcode'] ].iloc[0]['avg_ppm2']),
    axis=1) 

ndf



,customer_id,city,typology,subtypology,transaction_type,price,zipcode,bedrooms,bathrooms,area,area_land,is_professional,date_added,ppm2,over_valued,under_valued,normal
0,39323032373531333936323630,Barcelona Capital,Vivienda,Dúplex,sale,320000,08038,2,2,91,0,True,2012-05-18T12:26:00,3516.483516,1,0,0
3,39323032373531373435393038,Dosrius,Terreno,Terreno Urbanizable,sale,75000,08319,0,0,740,0,False,2014-12-11T00:01:00,101.351351,0,1,0
5,39323032373531323930363130,Navàs,Vivienda,Casa adosada,sale,99900,08670,5,1,220,0,True,2015-08-28T19:39:00,454.090909,0,1,0
6,39323032373531353539373530,Premià de Mar,Local comercial,NaN,sale,89000,08330,0,0,72,0,True,2016-03-09T18:09:00,1236.111111,0,1,0
7,39323032373532373237373132,Barcelona Capital,Vivienda,Piso,sale,2400000,08034,5,4,350,0,True,2016-05-31T14:13:00,6857.142857,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122369,393433303033303138363338,Pineda de Mar,Vivienda,Piso,sale,145000,08397,3,1,85,0,True,2019-12-05T17:47:00,1705.882353,1,0,0
122370,39323032373531333735313830,Olesa de Montserrat,Vivienda,Piso,sale,195000,08640,3,2,103,0,True,2019-12-05T20:47:00,1893.203883,1,0,0
122371,39323032373632343634333535,Montesquiu,Vivienda,Piso,sale,83400,08585,2,1,100,0,True,2019-12-06T06:16:00,834.000000,0,0,1
122372,39323032373631383234353237,L'Hospitalet de Llobregat,Vivienda,Ático,sale,385000,08902,3,2,132,0,True,2019-12-06T13:18:00,2916.666667,1,0,0


In [267]:
# which is the zipcode with the most number of bargains? (most under_valued count)

# To do this, once we have the over_valued, under_valued and normal values, we can group them and
# sort the dataframe by under_valued descending, and pick the first one's zipcode. That's easy.
ndf.groupby(['zipcode'])[['over_valued', 'under_valued', 'normal']].sum().sort_values(by=['under_valued'], ascending=False).index[0]

'08800'

In [268]:
# 8. Save the previous dataframe in a json file compressed as gzip
ndf.to_json('final_data.json', compression='gzip')